In [3]:
import psycopg2
import pandas_datareader as pdr
import pandas as pd
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib as plt
import seaborn as sns
sns.set()


The first thing I want to do is to make a dictionary of the datasets I'm interested in. I chose to analyze real estate data in the commercial and residential realm, as well as unemployment rate and cpi less food and energy. The reasoning here is according to Wilshire's fact sheet, they index companies that operate in both commercial and residential real estate, so I included those two; unemployment rate plus the cpi less F&E could help add general economic conditions to the model.

In [11]:
fred_dict = {
    're_comm': 'CREACBW027SBOG', # commercial real estate loans, weekly
    're_resd': 'RHEACBW027SBOG', # residential real estate loans, weekly
    'un_rate': 'UNRATE', # unemployment, monthly
    'cpi_less_fe': 'CORESTICKM159SFRBATL', # sticky price CPI, not counting food and energy, monthly
    're_wilshire': 'WILLREITIND' # the wilshire real estate market index, daily
}

# I chose to set the start dates for some of these series to earlier than exactly 5 years ago to hopefully 
# capture a little more data. Better to have to discard than extrapolate.

datasets = {
    're_comm': pdr.get_data_fred("CREACBW027SBOG",start='2017-10-14'), # commercial real estate loans, weekly
    're_resd': pdr.get_data_fred("RHEACBW027SBOG",start='2017-10-14'), # residential real estate loans, weekly
    'un_rate': pdr.get_data_fred("UNRATE",start='2017-10-01'), # unemployment, monthly
    'cpi_less_fe': pdr.get_data_fred("CORESTICKM159SFRBATL",start='2017-10-01'), # sticky price CPI, not counting food and energy, monthly
    're_wilshire': pdr.get_data_fred("WILLREITIND",start='2017-10-26') # the wilshire real estate market index, daily
}


            CREACBW027SBOG
DATE                      
2017-10-18       2068.3976
2017-10-25       2070.3496
2017-11-01       2071.9274
2017-11-08       2075.0345
2017-11-15       2078.4336
...                    ...
2022-09-14       2731.5585
2022-09-21       2735.3021
2022-09-28       2739.4757
2022-10-05       2746.6138
2022-10-12       2753.0907

[261 rows x 1 columns]
            RHEACBW027SBOG
DATE                      
2017-10-18        386.4911
2017-10-25        385.6180
2017-11-01        384.8354
2017-11-08        384.4569
2017-11-15        383.9686
...                    ...
2022-09-14        248.2911
2022-09-21        248.9592
2022-09-28        249.5233
2022-10-05        250.1710
2022-10-12        251.0011

[261 rows x 1 columns]
            UNRATE
DATE              
2017-10-01     4.2
2017-11-01     4.2
2017-12-01     4.1
2018-01-01     4.0
2018-02-01     4.1
2018-03-01     4.0
2018-04-01     4.0
2018-05-01     3.8
2018-06-01     4.0
2018-07-01     3.8
2018-08-01     3.8
201

Next I'll go ahead and resample these data. I'll fill in the `NAN`s by carrying the data points forward until a new data point is present. It's a little wacky but I didn't want to try to extrapolate and fill in too much missing data, so holding the less frequently updated sets constant seemed like a good option.

In [16]:
datasets['re_wilshire'] = datasets['re_wilshire'].ffill()
datasets_daily = {'re_wilshire_daily': datasets['re_wilshire']}
for dataset in datasets:
    if dataset != 're_wilshire':
        datasets_daily[f"{dataset}_daily"] = datasets[dataset].resample('D').ffill()

for dataset in datasets:
    print(datasets[dataset])

for dataset in datasets_daily:
    print(datasets_daily[dataset])


            CREACBW027SBOG
DATE                      
2017-10-18       2068.3976
2017-10-25       2070.3496
2017-11-01       2071.9274
2017-11-08       2075.0345
2017-11-15       2078.4336
...                    ...
2022-09-14       2731.5585
2022-09-21       2735.3021
2022-09-28       2739.4757
2022-10-05       2746.6138
2022-10-12       2753.0907

[261 rows x 1 columns]
            RHEACBW027SBOG
DATE                      
2017-10-18        386.4911
2017-10-25        385.6180
2017-11-01        384.8354
2017-11-08        384.4569
2017-11-15        383.9686
...                    ...
2022-09-14        248.2911
2022-09-21        248.9592
2022-09-28        249.5233
2022-10-05        250.1710
2022-10-12        251.0011

[261 rows x 1 columns]
            UNRATE
DATE              
2017-10-01     4.2
2017-11-01     4.2
2017-12-01     4.1
2018-01-01     4.0
2018-02-01     4.1
2018-03-01     4.0
2018-04-01     4.0
2018-05-01     3.8
2018-06-01     4.0
2018-07-01     3.8
2018-08-01     3.8
201

Setting up my sql tables here. I think sqlalchemy is working on a more automated way to map tables into SQL but here I'll just brute force it and create tables for both the original data sets and my resampled dataframes.

In [8]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import create_engine, Column, Integer, String, Float, DECIMAL, DateTime

Base = declarative_base()

class re_comm(Base):
    __tablename__ = 're_comm'
    id = Column(Integer, primary_key=True)
    date = Column('TimeSeries', DateTime, index=True)
    data = Column('CREACBW027SBOG', Float)

class re_resd(Base):
    __tablename__ = 're_resd'
    id = Column(Integer, primary_key=True)
    date = Column('TimeSeries', DateTime, index=True)
    data = Column('RHEACBW027SBOG', Float)

class un_rate(Base):
    __tablename__ = 'un_rate'
    id = Column(Integer, primary_key=True)
    date = Column('TimeSeries', DateTime, index=True)
    data = Column('UNRATE', Float)

class cpi_less_fe(Base):
    __tablename__ = 'cpi_less_fe'
    id = Column(Integer, primary_key=True)
    date = Column('TimeSeries', DateTime, index=True)
    data = Column('CORESTICKM159SFRBATL', Float)

class re_wilshire(Base):
    __tablename__ = 're_wilshire'
    id = Column(Integer, primary_key=True)
    date = Column('TimeSeries', DateTime, index=True)
    data = Column('WILLREITIND', DECIMAL)

class re_comm_daily(Base):
    __tablename__ = 're_comm_daily'
    id = Column(Integer, primary_key=True)
    date = Column('TimeSeries', DateTime, index=True)
    data = Column('CREACBW027SBOG_RS', Float)

class re_resd_daily(Base):
    __tablename__ = 're_resd_daily'
    id = Column(Integer, primary_key=True)
    date = Column('TimeSeries', DateTime, index=True)
    data = Column('RHEACBW027SBOG_RS', Float)

class un_rate_daily(Base):
    __tablename__ = 'un_rate_daily'
    id = Column(Integer, primary_key=True)
    date = Column('TimeSeries', DateTime, index=True)
    data = Column('UNRATE_RS', Float)

class cpi_less_fe_daily(Base):
    __tablename__ = 'cpi_less_fe_daily'
    id = Column(Integer, primary_key=True)
    date = Column('TimeSeries', DateTime, index=True)
    data = Column('CORESTICKM159SFRBATL_RS', Float)

with open('../.pgpass', 'r') as f:
    host, port, database, user, password = f.read().split(':')

engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}', echo=False)

Base.metadata.create_all(engine)

Next we will commit the original and resampled dataframes to SQL.

In [12]:
datasets['re_comm'].to_sql(name=re_comm.__tablename__, con=engine, if_exists='replace')
datasets['re_resd'].to_sql(name=re_resd.__tablename__, con=engine, if_exists='replace')
datasets['un_rate'].to_sql(name=un_rate.__tablename__, con=engine, if_exists='replace')
datasets['cpi_less_fe'].to_sql(name=cpi_less_fe.__tablename__, con=engine, if_exists='replace')
datasets['re_wilshire'].to_sql(name=re_wilshire.__tablename__, con=engine, if_exists='replace')
datasets_daily['re_comm_daily'].to_sql(name=re_comm_daily.__tablename__, con=engine, if_exists='replace')
datasets_daily['re_resd_daily'].to_sql(name=re_resd_daily.__tablename__, con=engine, if_exists='replace')
datasets_daily['un_rate_daily'].to_sql(name=un_rate_daily.__tablename__, con=engine, if_exists='replace')
datasets_daily['cpi_less_fe_daily'].to_sql(name=cpi_less_fe_daily.__tablename__, con=engine, if_exists='replace')

797